In [1]:
import pandas as pd
import pyodbc

# Import CSV
data = pd.read_csv (r'https://ihmecovid19storage.blob.core.windows.net/latest/data_download_file_worse_2021.csv', parse_dates=['date'])   
df = pd.DataFrame(data)
df.head(5)

,location_id,date,version_name,location_name,inf_mean,inf_upper,inf_lower,seir_cumulative_mean,seir_cumulative_lower,seir_cumulative_upper,...,icu_beds_upper,icu_beds_lower,admis_mean,admis_upper,admis_lower,all_bed_capacity,icu_bed_capacity,infection_fatality,infection_detection,infection_hospitalization
0,1,2021-01-01,worse,Global,5828875.73,6895980.81,5122747.56,4264127.44,3094077.60,6153273.98,...,325101.04,179093.58,92310.81,109957.99,76742.27,17518594.0,536960.0,0.0038,0.1054,0.0153
1,1,2021-01-02,worse,Global,5788277.24,6852383.26,5089471.72,4287293.48,3111659.93,6185739.18,...,329935.13,180165.03,92542.59,110317.10,76882.27,17518594.0,536960.0,0.0038,0.1085,0.0154
2,1,2021-01-03,worse,Global,5743406.34,6777639.81,5052249.54,4310766.96,3129492.31,6218601.96,...,334392.64,181138.82,92732.48,110612.07,76985.22,17518594.0,536960.0,0.0039,0.1117,0.0154
3,1,2021-01-04,worse,Global,5694163.41,6698841.35,4999227.57,4334564.22,3147586.63,6251887.72,...,338928.64,182237.60,92859.14,110821.50,77030.03,17518594.0,536960.0,0.0039,0.1148,0.0155
4,1,2021-01-05,worse,Global,5640593.37,6616390.57,4941048.77,4358692.51,3165947.82,6285610.64,...,343383.47,183200.10,92908.79,110930.87,77005.14,17518594.0,536960.0,0.0040,0.1177,0.0155


In [8]:
df.insert(0, 'New_ID', range(101, 101 + len(df)))
df.head()

,New_ID,location_id,date,version_name,location_name,inf_mean,inf_upper,inf_lower,seir_cumulative_mean,seir_cumulative_lower,...,icu_beds_upper,icu_beds_lower,admis_mean,admis_upper,admis_lower,all_bed_capacity,icu_bed_capacity,infection_fatality,infection_detection,infection_hospitalization
0,101,1,2021-01-01,worse,Global,5828875.73,6895980.81,5122747.56,4264127.44,3094077.60,...,325101.04,179093.58,92310.81,109957.99,76742.27,17518594.0,536960.0,0.0038,0.1054,0.0153
1,102,1,2021-01-02,worse,Global,5788277.24,6852383.26,5089471.72,4287293.48,3111659.93,...,329935.13,180165.03,92542.59,110317.10,76882.27,17518594.0,536960.0,0.0038,0.1085,0.0154
2,103,1,2021-01-03,worse,Global,5743406.34,6777639.81,5052249.54,4310766.96,3129492.31,...,334392.64,181138.82,92732.48,110612.07,76985.22,17518594.0,536960.0,0.0039,0.1117,0.0154
3,104,1,2021-01-04,worse,Global,5694163.41,6698841.35,4999227.57,4334564.22,3147586.63,...,338928.64,182237.60,92859.14,110821.50,77030.03,17518594.0,536960.0,0.0039,0.1148,0.0155
4,105,1,2021-01-05,worse,Global,5640593.37,6616390.57,4941048.77,4358692.51,3165947.82,...,343383.47,183200.10,92908.79,110930.87,77005.14,17518594.0,536960.0,0.0040,0.1177,0.0155


In [33]:
len(list(df.columns))

61

In [3]:
df.date.dtype

dtype('<M8[ns]')

In [21]:
string1 = ""
for i in df.columns:
    print(i, df[i].dtype)

New_ID int32
location_id int64
date datetime64[ns]
version_name object
location_name object
inf_mean float64
inf_upper float64
inf_lower float64
seir_cumulative_mean float64
seir_cumulative_lower float64
seir_cumulative_upper float64
inf_cuml_mean float64
inf_cuml_lower float64
inf_cuml_upper float64
seir_daily_mean float64
seir_daily_lower float64
seir_daily_upper float64
seir_daily_unscaled_mean float64
seir_daily_unscaled_lower float64
seir_daily_unscaled_upper float64
seir_cumulative_unscaled_mean float64
seir_cumulative_unscaled_lower float64
seir_cumulative_unscaled_upper float64
reff_mean float64
reff_lower float64
reff_upper float64
cumulative_deaths float64
daily_deaths float64
cumulative_cases float64
cumulative_deaths_unscaled float64
cumulative_hospitalizations float64
daily_deaths_unscaled float64
daily_infections float64
population float64
mobility_mean float64
mobility_obs int64
testing_mean float64
testing_lower float64
testing_upper float64
testing_obs int64
pneumonia_

In [4]:
# Connect to SQL Server
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=MONTXPS\COVID_PROJECT_SQ;'
                      'Database=COVID_project;'
                      'Trusted_Connection=yes;')
cursor = conn.cursor()

In [27]:
# Create Table
cursor.execute('''
		CREATE TABLE data_download_file_worse_2021 (
			New_ID int primary key,
			location_id int,
			date datetime,
			version_name varchar,
			location_name varchar,
			inf_mean float,
			inf_upper float,
			inf_lower float,
			seir_cumulative_mean float,
			seir_cumulative_lower float,
			seir_cumulative_upper float,
			inf_cuml_mean float,
			inf_cuml_lower float,
			inf_cuml_upper float,
			seir_daily_mean float,
			seir_daily_lower float,
			seir_daily_upper float,
			seir_daily_unscaled_mean float,
			seir_daily_unscaled_lower float,
			seir_daily_unscaled_upper float,
			seir_cumulative_unscaled_mean float,
			seir_cumulative_unscaled_lower float,
			seir_cumulative_unscaled_upper float,
			reff_mean float,
			reff_lower float,
			reff_upper float,
			cumulative_deaths float,
			daily_deaths float,
			cumulative_cases float,
			cumulative_deaths_unscaled float,
			cumulative_hospitalizations float,
			daily_deaths_unscaled float,
			daily_infections float,
			population float,
			mobility_mean float,
			mobility_obs int,
			testing_mean float,
			testing_lower float,
			testing_upper float,
			testing_obs int,
			pneumonia_mean float,
			pneumonia_obs int,
			mask_use_mean float,
			mask_use_obs int,
			cumulative_all_vaccinated float,
			cumulative_all_fully_vaccinated float,
			cumulative_all_effectively_vaccinated float,
			hospital_beds_mean float,
			hospital_beds_upper float,
			hospital_beds_lower float,
			icu_beds_mean float,
			icu_beds_upper float,
			icu_beds_lower float,
			admis_mean float,
			admis_upper float,
			admis_lower float,
			all_bed_capacity float,
			icu_bed_capacity float,
			infection_fatality float,
			infection_detection float,
			infection_hospitalization float,
			)
               ''')

In [29]:
conn.commit()

In [35]:
# Insert DataFrame to Table
for row in df.itertuples():
    cursor.execute(f'''
                INSERT INTO data_download_file_worse_2021 ('New_ID',
                                    'location_id',
                                    'date',
                                    'version_name',
                                    'location_name',
                                    'inf_mean',
                                    'inf_upper',
                                    'inf_lower',
                                    'seir_cumulative_mean',
                                    'seir_cumulative_lower',
                                    'seir_cumulative_upper',
                                    'inf_cuml_mean',
                                    'inf_cuml_lower',
                                    'inf_cuml_upper',
                                    'seir_daily_mean',
                                    'seir_daily_lower',
                                    'seir_daily_upper',
                                    'seir_daily_unscaled_mean',
                                    'seir_daily_unscaled_lower',
                                    'seir_daily_unscaled_upper',
                                    'seir_cumulative_unscaled_mean',
                                    'seir_cumulative_unscaled_lower',
                                    'seir_cumulative_unscaled_upper',
                                    'reff_mean',
                                    'reff_lower',
                                    'reff_upper',
                                    'cumulative_deaths',
                                    'daily_deaths',
                                    'cumulative_cases',
                                    'cumulative_deaths_unscaled',
                                    'cumulative_hospitalizations',
                                    'daily_deaths_unscaled',
                                    'daily_infections',
                                    'population',
                                    'mobility_mean',
                                    'mobility_obs',
                                    'testing_mean',
                                    'testing_lower',
                                    'testing_upper',
                                    'testing_obs',
                                    'pneumonia_mean',
                                    'pneumonia_obs',
                                    'mask_use_mean',
                                    'mask_use_obs',
                                    'cumulative_all_vaccinated',
                                    'cumulative_all_fully_vaccinated',
                                    'cumulative_all_effectively_vaccinated',
                                    'hospital_beds_mean',
                                    'hospital_beds_upper',
                                    'hospital_beds_lower',
                                    'icu_beds_mean',
                                    'icu_beds_upper',
                                    'icu_beds_lower',
                                    'admis_mean',
                                    'admis_upper',
                                    'admis_lower',
                                    'all_bed_capacity',
                                    'icu_bed_capacity',
                                    'infection_fatality',
                                    'infection_detection',
                                    'infection_hospitalization')
                VALUES ('?')
                ''',
                row.New_ID,
                row.location_id,
                row.date,
                row.version_name,
                row.location_name,
                row.inf_mean,
                row.inf_upper,
                row.inf_lower,
                row.seir_cumulative_mean,
                row.seir_cumulative_lower,
                row.seir_cumulative_upper,
                row.inf_cuml_mean,
                row.inf_cuml_lower,
                row.inf_cuml_upper,
                row.seir_daily_mean,
                row.seir_daily_lower,
                row.seir_daily_upper,
                row.seir_daily_unscaled_mean,
                row.seir_daily_unscaled_lower,
                row.seir_daily_unscaled_upper,
                row.seir_cumulative_unscaled_mean,
                row.seir_cumulative_unscaled_lower,
                row.seir_cumulative_unscaled_upper,
                row.reff_mean,
                row.reff_lower,
                row.reff_upper,
                row.cumulative_deaths,
                row.daily_deaths,
                row.cumulative_cases,
                row.cumulative_deaths_unscaled,
                row.cumulative_hospitalizations,
                row.daily_deaths_unscaled,
                row.daily_infections,
                row.population,
                row.mobility_mean,
                row.mobility_obs,
                row.testing_mean,
                row.testing_lower,
                row.testing_upper,
                row.testing_obs,
                row.pneumonia_mean,
                row.pneumonia_obs,
                row.mask_use_mean,
                row.mask_use_obs,
                row.cumulative_all_vaccinated,
                row.cumulative_all_fully_vaccinated,
                row.cumulative_all_effectively_vaccinated,
                row.hospital_beds_mean,
                row.hospital_beds_upper,
                row.hospital_beds_lower,
                row.icu_beds_mean,
                row.icu_beds_upper,
                row.icu_beds_lower,
                row.admis_mean,
                row.admis_upper,
                row.admis_lower,
                row.all_bed_capacity,
                row.icu_bed_capacity,
                row.infection_fatality,
                row.infection_detection,
                row.infection_hospitalization
                )
conn.commit()

ProgrammingError: ('The SQL contains 0 parameter markers, but 61 parameters were supplied', 'HY000')